# EQness EEG lab

EEG system:
- Ultracortex
- 16 EEG channels
- 3 Accelerator channels

In [ ]:
import mne
from mne.preprocessing import ICA

In [ ]:
cytondaisy_montage = ['Fp1', 'Fp2', 'C3', 'C4', 'T5', 'T6', 'O1', 'O2', 'F7', 'F8', 'F3', 'F4', 'T3', 'T4', 'P3', 'P4']
mapping = { f'EEG {i+1:d}' : ch for i, ch in enumerate(cytondaisy_montage) }
print("mapping:\n", mapping)

In [ ]:
f = "./data/OpenBCI-BDF-2024-08-30_16-19-18.bdf"
raw = mne.io.read_raw_bdf(f, preload=True, verbose=False)
print(raw.info)
print("[+] All channels in recording:")
raw.drop_channels(['Accel X', 'Accel Y', 'Accel Z'])
print("\n[+] After removing Acceleration channels")
mne.rename_channels(raw.info, mapping)
montage=mne.channels.make_standard_montage("standard_1020") # standard_1005
raw.set_montage(montage, verbose=False)
# frequency boundaries
freq_lo = 1
freq_hi = 25
raw.filter(l_freq=freq_lo, h_freq=freq_hi, verbose=False)
raw.plot()

### Select data segment

In [ ]:
t0, t1 = 1167, 1296
print(f"Segment length: {(t1-t0)/60:.1f} min")
raw_segment = raw.copy().crop(tmin=t0, tmax=t1)

In [ ]:
raw_segment.compute_psd(fmin=freq_lo, fmax=freq_hi).plot()

### ICA artefact correction

In [ ]:
%matplotlib qt
ica_segment = ICA(n_components=12, max_iter="auto", random_state=97)
ica_segment.fit(raw_segment)
ica_segment.plot_sources(raw_segment, show_scrollbars=False)
ica_segment.plot_properties(raw_segment)

In [ ]:
raw_segment_recon = raw_segment.copy()
ica_segment.apply(raw_segment_recon)
raw_segment_recon.plot()